In [1]:
# [Set up function]
import numpy as np
import pandas as pd

def comfa_oa_compute(input_df: pd.DataFrame, output_path: str = None):
    ###################Set up input parameters################
    # --- helpers ---
    # Vr - Xiaoyu Li's COMFA-O method (Mathematical Expectation for 0-360 wind)
    def Vr_function(Vac, Ws):
        """
        Vectorized approximation of:
        mapply(function(Vac_i, Ws_i) {
          integrand <- function(theta) sqrt(Vac_i^2 + Ws_i^2 - 2 * Vac_i * Ws_i * cos(theta))
          (1 / (2 * pi)) * integrate(integrand, lower = 0, upper = 2 * pi)$value
        }, Vac, Ws)
        """
        Vac = np.asarray(Vac, dtype=float)
        Ws  = np.asarray(Ws,  dtype=float)
        # numerical expectation over theta in [0, 2π]
        n_theta = 720
        theta = np.linspace(0.0, 2.0*np.pi, n_theta, endpoint=True)
        # Broadcast to (n, n_theta)
        Vac_b = Vac[:, None]
        Ws_b  = Ws[:, None]
        integrand = np.sqrt(np.maximum(0.0, Vac_b**2 + Ws_b**2 - 2.0*Vac_b*Ws_b*np.cos(theta)[None, :]))
        # Trapezoidal numeric integration average over 0..2π
        area = np.trapz(integrand, theta, axis=1)
        return (1.0 / (2.0*np.pi)) * area

    # [Purpose] tsf (clothes surface temperature) ~ (METc,Ta,Pv) Ominvar method (only used for basic sweat)
    # [目的] 皮肤表面温度 (仅用于估算基础出汗)
    def Tsf_Ominivar_function(METc, Pv, Ta):
        METc = np.asarray(METc, dtype=float)
        Pv   = np.asarray(Pv,   dtype=float)
        Ta   = np.asarray(Ta,   dtype=float)
        return 35.7 - 0.028 * (METc) - 0.155 * (
            (METc) - 3.05e-3 * (5733 - 6.99 * (METc) - 1000 * Pv)
            - 0.42 * ((METc) - 58.15) - 1.7e-5 * METc * (5867 - 1000 * Pv)
            - 0.0014 * METc * (34 - Ta)
        )
    ###################Set up input parameters finished#######

    # ----- Internal core calculator so we can reuse it for CET -----
    def compute_core(df: pd.DataFrame) -> pd.DataFrame:
        # Input:
        Ta          = df["Ta"].to_numpy(dtype=float)            # Celsius
        Rh          = df["Rh"].to_numpy(dtype=float)            # %
        clo         = df["clo"].to_numpy(dtype=float)           # clo
        Tmrt        = df["Tmrt"].to_numpy(dtype=float)          # Celsius
        METs        = df["METs"].to_numpy(dtype=float)          # in met
        Age         = df["Age"].to_numpy(dtype=float)
        sex_raw     = df["sex"]                                 # "male" = 1, "female" = 2
        Ws          = df["Ws"].to_numpy(dtype=float)            # m/s
        Height      = df["Height"].to_numpy(dtype=float)        # meter
        Weight      = df["Weight"].to_numpy(dtype=float)        # kg
        bodyPosture = df["bodyPosture"].astype(str)             # "sitting" or "standing" or "crouching", default value = "standing"

        # Input data correction part
        # ---- enforce minimum clothes insulation ----
        clo_min = 0.01   # example floor; adjust (e.g., 0.30 for sports, 0.50 for indoor summer tests)
        df["clo_raw"] = df["clo"]
        # Replace NAs with the floor, then clamp all values to at least CLO_MIN
        clo = df["clo"].copy()
        clo = clo.fillna(clo_min)
        clo = np.maximum(clo.to_numpy(dtype=float), clo_min)
        df["clo"] = clo
        clo = df["clo"].to_numpy(dtype=float)
        # ---- enforce minimum wind speed (Ws) >= 0.1 m/s ----
        ws_min = 0.1
        df["Ws_raw"] = df["Ws"]
        Ws = df["Ws"].copy().fillna(ws_min).to_numpy(dtype=float)
        Ws = np.maximum(Ws, ws_min)
        df["Ws"] = Ws
        Ws = df["Ws"].to_numpy(dtype=float)

        #####################COMFA_OA Calculation##############################
        # Stefan-Boltzmann constant
        kB = 5.67e-08

        # Radiation efficiency [辐射效率]
        bp = np.char.lower(bodyPosture.to_numpy(dtype=str))
        rad_eff = np.where(
            pd.isna(bodyPosture),
            0.895,
            np.where(bp == 'sitting', 0.691,
                     np.where(bp == 'standing', 0.895,
                              np.where(bp == 'crouching', 0.6, 0.895)))
        )

        # Barometric pressure [大气压力]
        pBaroHPA_setting = 1013.25
        pBaroHPA = 1013.25

        # Vapour pressure
        # [Unit] kPa, Tetens formula
        Pv = ((0.6108 * (np.exp((17.269 * Ta) / (Ta + 237.3)))) * (Rh / 100.0))

        # Revised Harris-benedict
        sex_char = pd.Series(sex_raw).astype(str).str.lower()
        sex_is_male = sex_char.isin(["male", "m"]).to_numpy() | pd.Series(sex_raw).astype(str).isin(["1"]).to_numpy() | pd.to_numeric(pd.Series(sex_raw), errors="coerce").eq(1).fillna(False).to_numpy()
        sex_std = np.where(sex_is_male, 1, 2)

        rmr = np.where(
            sex_is_male,
            9.65 * Weight + 573 * Height - 5.8 * Age + 260,
            7.38 * Weight + 607 * Height - 2.31 * Age + 43
        )
        Q_met_min = rmr / 24.0 * 1.163  # Q_met_min: minimum meta heat by RMR

        ### Optimized Kozey formula
        VO2_original  = 3.5
        VO2_corrected = rmr / 1440.0 / 5.0 / Weight * 1000.0  # Harris-Benedict predicted RMR, Kcal/day to ml·kg⁻¹·min⁻¹
        METc = 1.0 + (METs - 1.0) * (VO2_original / VO2_corrected)

        # BSA: 3D scan 2010
        BSA = np.where(
            sex_is_male,
            79.8106 * (Height * 100.0)**0.7271 * (Weight**0.3980) / 10000.0,
            84.4673 * (Height * 100.0)**0.6997 * (Weight**0.4176) / 10000.0
        )

        # Relative speed function
        # Effect of temperature and clothing thermal resistance on human sweat at low activity levels
        Vac = np.where(METs >= 1.5, 0.0052 * (METs * 58.15 - 58.15 * 1.5), 0.0)  # activity speed

        # Vr - Xiaoyu Li's COMFA-O method (Mathematical Expectation for 0-360 wind)
        Vr = Vr_function(Vac, Ws)

        ###  get the data from clo
        Rco  = 0.155 * clo / 0.082 * 100.0
        Rcvo = Rco * 2.74524064171123

        Trans = 100.0  # Transmissivity (%)
        # Clothing Vapour Resistance, from human clothing table
        Rcv = Rcvo * (-0.8 * (1.0 - np.exp(-Vr / 1.095)) + 1.0)
        # Clothing resistance, from energy table, Row AH
        Rc = Rco * (-0.37 * (1.0 - np.exp(-Vr / 0.72)) + 1.0)

        # M: Metabolic energy(W/m2)
        # Heat loss consumed through breathing (%)
        f = (0.15 - (0.0173 * Pv)) - (0.0014 * Ta)

        ### Qmet, avoid underestimation of Q_met by applying the RMR as minimum
        Q_met_raw = (1.0 - f) * (rmr / 24.0 * 1.163) * (METc * 58.15) / 80.0
        Q_met = np.where(Q_met_raw <= Q_met_min, Q_met_min, Q_met_raw)

        ##################COMFA-O Tmrt Method#######################
        # COMFA - kabs
        # 0.98 is the skin emissivity
        kabs_labs_totl = (Tmrt + 273.15)**4 * (5.67e-8 * 0.98)

        # COMFA - Rabs
        Rabs = kabs_labs_totl * rad_eff

        # Tc: core temperature
        Tc = np.where(
            Age < 20.0,
            36.5 + (0.0043 * Q_met),
            np.where(
                sex_is_male,
                (36.5 + (0.0043 * Q_met)) - 0.0060 * (Age - 20.0),    ###0.0060
                (36.5 + (0.0043 * Q_met)) - 0.0066 * (Age - 20.0)     ###0.0066
            )
        )

        # [Purpose] Tsk (skin temperature) ~ (Ta,Tmrt,Pv,Vr,METc,Tc)
        # New Tsk formula
        Tsk_high = 12.2 + 0.020 * Ta + 0.044 * Tmrt + 0.194 * Pv - 0.253 * Vr + 0.00297 * METc * 58.15 + 0.513 * Tc
        Tsk_low  = 7.2  + 0.064 * Ta + 0.061 * Tmrt + 0.198 * Pv - 0.348 * Vr + 0.616 * Tc
        Tsk_mid  = (7.2 + 0.064 * Ta + 0.061 * Tmrt + 0.198 * Pv - 0.348 * Vr + 0.616 * Tc) + \
                   2.5 * ((12.2 + 0.020 * Ta + 0.044 * Tmrt + 0.194 * Pv - 0.253 * Vr + 0.513 * Tc) -
                          (7.2 + 0.064 * Ta + 0.061 * Tmrt + 0.198 * Pv - 0.348 * Vr + 0.616 * Tc)) * (clo - 0.2)
        Tsk = np.where(clo >= 0.6, Tsk_high, np.where(clo > 0.2, Tsk_mid, Tsk_low))

        # TRemitted: The emitted terrestrial radiation (W/m2)
        Re = 11333.0 * Vr
        # Ra is
        Ra = np.where(
            Re > 40000.0, (0.17 / ((0.71**0.33) * 0.000022 * 0.0266 * (Re**0.805))),
            np.where(
                Re < 4000.0, (0.17 / ((0.71**0.33) * 0.000022 * 0.683 * (Re**0.466))),
                (0.17 / ((0.71**0.33) * 0.000022 * 0.193 * (Re**0.618)))
            )
        )

        #############################Sweat heat loss##################
        # facl, in decimal %, percentage of body covered by clothes
        f_clo_area = (173.51 * clo - 2.36 - 100.76 * clo * clo + 19.28 * (clo**3.0)) / 100.0
        f_clo_area = np.where(f_clo_area >= 1.0, 1.0, f_clo_area)
        # To address <0 value for extreme low clo condition
        f_clo_area = np.maximum(f_clo_area, 0.0)

        # fcl, Spasic’s expansion formula, According to ISO 110793) and ISO 79331) (based on McCullough et al.9))
        fCloAreaExpansion = 1.0 + (0.31 * clo)

        areaClo  = BSA * f_clo_area + BSA * (fCloAreaExpansion - 1.0)
        areaNude = BSA * (1.0 - f_clo_area)

        # clothes surface temperature [Purpose: COMFA convective heat transfer model]
        # 服装表面温度 [目的] 用于COMFA 对流换热模型
        # Edit 2025 01 24,
        Tsf = (((Tsk - Ta) / (Rc + Ra)) * Ra) + Ta

        # convective heat transfer coefficients for free convection modes
        # 对流换热系数 （自由对流）
        # NOTE: fractional power of negative base yields NaN; mirror R behavior
        delta = Tsf - Ta
        h_c_free = np.empty_like(delta)
        mask_pos = delta >= 0.0
        h_c_free[mask_pos] = 2.38 * (delta[mask_pos]**0.25)
        h_c_free[~mask_pos] = np.nan
        # Set the NAN value to -999 for the h_c_free
        h_c_free = np.where(np.isnan(h_c_free), -999.0, h_c_free)

        # Convective heat transfer coefficient (forced convection)
        # 对流换热系数（强制对流）
        # convective heat transfer coefficients for forced convection
        h_c_forced = 2.67 + 6.5 * (Vr**0.67)
        h_c_forced = h_c_forced * (pBaroHPA / pBaroHPA_setting)**0.55

        # Final hConv
        # compare free vs. forced and take the bigger value as the convective heat transfer coefficient
        hConv = np.maximum(h_c_free, h_c_forced)
        hConv = (2.67 + 6.5 * (Vr**0.67)) * (pBaroHPA / pBaroHPA_setting)**0.55

        # Convection
        Q_conv_bare = hConv * (Ta - Tsk) * areaNude
        Q_conv_clo  = hConv * (Ta - Tsf) * areaClo
        Conv = -(Q_conv_bare + Q_conv_clo) / BSA

        # Radeff method (SELECTED)
        TRemitted = rad_eff * ((0.95 * (kB) * ((Tsk + 273.15)**4)))

        # [Purpose] basal sweat estimation (non-activity triggered sweat)
        # [目的] 基础出汗估算 (非运动刺激产生的出汗)
        # [Method] the sweat estimation by E_req and E_max, Gonzalez et al. adjusted by Omidvar
        Tsf_Ominivar = Tsf_Ominivar_function(METc, Pv, Ta)  # Clothes surface temperature, Ominivar method
        # Clothes surface temperature when met = 4
        T_cl = 29.187 - clo * (17.447 + 0.0011 * Pv * 1000.0 + 0.0504 * Ta)
        # Heat conduction coefficient
        h_cl = 6.45 / clo

        # Clothed vs. nude ratio
        # Clothed vs. nude ratio
        f_cl = np.where(clo < 0.5, 1.0 + 0.2 * clo, 1.05 + 0.1 * clo)

        # convective heat transfer coefficients for forced convection
        # Maximum evaporation limited by environmental parameters when met =4
        E_max = (16.7 + 0.371 * clo * (h_cl**2)) * (4.13 - 0.001 * Pv * 1000.0)

        # Required evaporation by energy balance when met =4
        E_req = 232.6 - f_cl * hConv * (T_cl - Ta) - 3.96 * 10**(-8) * f_cl * (
            (T_cl + 273.15)**4 - (Tmrt + 273.15)**4
        )

        # Es_b: basal sweat rate before adjustment
        # 基础出汗速率(调整前)
        Es_b = 36.75 + 0.3818 * E_req - 0.2175 * E_max
        Es_b = np.where(E_req <= 0.0, 0.0, Es_b)

        # Beta coefficient to correct the basal sweat rate
        # Beta系数用于调整基础出汗率
        beta_sweat = np.where(
            Es_b >= 200.0, 1.0,
            np.where(Es_b > 150.0, 0.5,
                     np.where(Es_b >= 0.0, 0.333, 0.0))
        )

        # Final: Total sweat heat loss calculation
        ## Basal sweat heat loss (environmental triggered)
        ## 由环境因素导致的基础出汗量
        q_sweat_basal = np.where(
            Age >= 20.0,
            (0.675 * beta_sweat / ((1.0 + 0.155 * clo * (4.6 + hConv + 0.046 * Tmrt))) * Es_b) * (1.0 - (0.005 * (Age - 20.0))),
            (0.675 * beta_sweat / ((1.0 + 0.155 * clo * (4.6 + hConv + 0.046 * Tmrt))) * Es_b)
        )

        #############################################################################################################################
        ## Define basal sweat initialization point
        # When the net budget is less than 20 w/m2, we assume no basal sweat here
        q_sweat_basal = np.where((Q_met + Rabs - Conv - TRemitted) < 20.0, 0.0, q_sweat_basal)

        q_sweat_act = np.where(
            Age >= 20.0,
            0.42 * (Q_met - 58.15) * (1.0 - (0.005 * (Age - 20.0))),
            0.42 * (Q_met - 58.15)
        )
        # basal sweat always >= 0
        q_sweat_act = np.where(q_sweat_act <= 0.0, 0.0, q_sweat_act)

        ## Evaporative heat loss through sweat
        ## 出汗导致的总蒸发散热
        Es = q_sweat_act + q_sweat_basal
        Es = np.where(E_req <= 0.0, 0.0, Es)

        # Specific humidity at skin
        qs = 0.6108 * (np.exp((17.269 * Tsk) / (Tsk + 237.3)))
        ############Evap: The evaporative heat loss (W/m2)##################
        # [Purpose] Evaporative heat loss limited by environmental maximum evap ability
        Ei = (1.16 * 2442.0) * ((qs - Pv) / (7700.0 + Rcv + (0.92 * Ra)))  # Evaporative heat loss through diffusion (invisible)
        Em = (1.16 * 2442.0) * ((qs - Pv) / (Rcv + (0.92 * Ra)))           # Maximal evaporative heat loss

        Evap = Es + Ei
        Evap = np.where(Evap < Em, Evap, Em)
        ##############################################################################################################
        ##############################################################################################################

        BMI = Weight / (Height**2)  # height in m, Weight in kg

        BF = np.where(
            sex_is_male,  (1.39 * BMI) + (0.16 * Age) - 19.34,
                          (1.39 * BMI) + (0.16 * Age) - 9.0
        )

        Tc_diff  = np.where(Tc <= 37.0, 37.0 - Tc, 0.0)
        Tsk_diff = np.where(Tsk <= 33.0, 33.0 - Tsk, 0.0)
        # Set the threshold of shivering (When COMFA < -20)
        # when the net COMFA-OA < -15, start to shivering
        cold_signal = np.where((Q_met + Rabs - Conv - Evap - TRemitted) < (-15.0), 1.0, 0.0)

        Qshiv = cold_signal * (155.5 * Tc_diff + 47.0 * Tsk_diff - 1.57 * (Tsk_diff**2)) / (BF**0.5)

        # --- compute COMFA-OA (as a valid R object name) ---
        comfa_oa = Q_met + Rabs - Conv - Evap - TRemitted + Qshiv

        return pd.DataFrame({"COMFA_OA": comfa_oa})

    # ---------- Compute COMFA_OA for the provided data ----------
    df_main = input_df.copy()
    out_core = compute_core(df_main)
    comfa_vec = out_core["COMFA_OA"].to_numpy()

    # ---------- CET: solve for Ta that reproduces COMFA_OA under the fixed indoor reference ----------
    # CET definition:
    # ---- robust root finder: safe bracketing without NA in while/if conditions ----
    # ---------- CET: solve for Ta that reproduces COMFA_OA under UTCI-style reference ----------
    # Reference condition: Tmrt = Ta, Ws = 0.3 m/s, Rh = 50%, clo = 0.9, sex = 1, Height = 1.75, Weight = 75,
    # Age = 65, METs = 2.3, bodyPosture = "standing"

    ref_constants = dict(
        Rh = 50.0, clo = 0.9, sex = 1, Height = 1.75, Weight = 75.0,
        Age = 65.0, METs = 2.3, bodyPosture = "standing",
        Ws = 0.3  # <-- use ~0.3 m/s at body height, 0.5m for 10m ws
    )

    # Build a template DF for the reference condition (one row; we'll vary Ta; Tmrt = Ta)
    def make_ref_df(Ta_guess: float, n: int = 1) -> pd.DataFrame:
        return pd.DataFrame({
            "Ta":          np.repeat(Ta_guess, n),
            "Rh":          np.repeat(ref_constants["Rh"], n),
            "clo":         np.repeat(ref_constants["clo"], n),
            "Tmrt":        np.repeat(Ta_guess, n),                # <-- UTCI: Tmrt equals Ta
            "METs":        np.repeat(ref_constants["METs"], n),
            "Age":         np.repeat(ref_constants["Age"], n),
            "sex":         np.repeat(ref_constants["sex"], n),
            "Ws":          np.repeat(ref_constants["Ws"], n),     # <-- fixed 0.3 m/s
            "Height":      np.repeat(ref_constants["Height"], n),
            "Weight":      np.repeat(ref_constants["Weight"], n),
            "bodyPosture": np.repeat(ref_constants["bodyPosture"], n),
        })

    # ---- robust root finder: safe bracketing without NA in while/if conditions ----
    def solve_cet_for_target(target_comfa: float, ta_center: float = 21.0, init_width: float = 20.0, max_expand: int = 5):
        def f(ta_scalar: float):
            ref_df = make_ref_df(float(ta_scalar), n=1)
            return float(compute_core(ref_df)["COMFA_OA"].iloc[0] - target_comfa)

        # start with a bracket around ta_center
        low  = float(ta_center - init_width)
        high = float(ta_center + init_width)

        def eval_f(x):
            try:
                val = f(x)
                if np.isfinite(val):
                    return val
                return np.nan
            except Exception:
                return np.nan

        f_low  = eval_f(low)
        f_high = eval_f(high)

        # expand bracket a few times if needed (avoid NA in logical checks)
        expanded = 0
        sign_change = np.isfinite(f_low) and np.isfinite(f_high) and (np.sign(f_low) != np.sign(f_high))

        while (not sign_change) and (expanded < max_expand):
            low  -= init_width
            high += init_width
            f_low  = eval_f(low)
            f_high = eval_f(high)
            sign_change = np.isfinite(f_low) and np.isfinite(f_high) and (np.sign(f_low) != np.sign(f_high))
            expanded += 1

        if not sign_change:
            # final fallback: coarse scan to try to locate a sign change
            grid = np.arange(low, high + 1.0, 1.0, dtype=float)
            vals = np.array([eval_f(x) for x in grid], dtype=float)
            ok = np.where(np.isfinite(vals))[0]
            if ok.size >= 2:
                # find adjacent sign change
                signs = np.sign(vals[ok])
                diffs = signs[1:] != signs[:-1]
                idxs = np.where(diffs)[0]
                if idxs.size >= 1:
                    i1 = ok[idxs[0]]
                    i2 = ok[idxs[0] + 1]
                    low, high = grid[i1], grid[i2]
                    f_low, f_high = vals[i1], vals[i2]
                    sign_change = True

        if not sign_change:
            return np.nan

        # now safe to call a simple bisection (uniroot-like)
        # tol and maxiter similar to R's uniroot usage
        tol = 1e-4
        maxiter = 200
        a, b = low, high
        fa, fb = f_low, f_high
        # Ensure a < b and fa, fb have opposite signs
        if a > b:
            a, b = b, a
            fa, fb = fb, fa

        # Bisection
        for _ in range(maxiter):
            mid = 0.5 * (a + b)
            fm = eval_f(mid)
            if not np.isfinite(fm):
                # nudge slightly if NaN encountered
                mid = np.nextafter(mid, b)
                fm = eval_f(mid)
                if not np.isfinite(fm):
                    break
            if abs(fm) < tol or (b - a) < tol:
                return mid
            # choose subinterval
            if np.sign(fa) != np.sign(fm):
                b, fb = mid, fm
            else:
                a, fa = mid, fm
        return 0.5 * (a + b)

    # vectorize CET over rows; use observed Ta as center to help convergence
    ta_center_vec = df_main["Ta"].to_numpy(dtype=float)
    cet_vals = np.array([
        solve_cet_for_target(target_comfa=tc, ta_center=tc0 if np.isfinite(tc0) else 21.0)
        for tc, tc0 in zip(comfa_vec, ta_center_vec)
    ], dtype=float)

    # --- append to input and (optionally) write CSV ---
    out = df_main.copy()
    out["COMFA_OA"] = comfa_vec
    out["CET"] = cet_vals.astype(float)

    if output_path is not None:
        out.to_csv(output_path, index=False)
    return out


# ---- example (uncomment to run) ----
input_df = pd.read_csv("./2_COMFA_OA_example_input.csv")
output_df = comfa_oa_compute(input_df, output_path="./2_COMFA_OA_output_py.csv")

